<a href="https://colab.research.google.com/github/Nayim-Imrit/englistToFrenchTranslationGRU/blob/main/Translation(english_to_french)EncoderDecoderAndAccuracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%tensorflow_version 2.x
import collections

import helper
import numpy as np

from tensorflow.keras import Input, Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, LSTM
from keras.layers.embeddings import Embedding
from tensorflow.keras.optimizers import Adam # - Works
from keras.losses import sparse_categorical_crossentropy

In [4]:
import os
 
 
def load_data(path):
    """
    Load dataset
    """
    input_file = os.path.join(path)
    with open(input_file, "r") as f:
        data = f.read()
 
    return data.split('\n')

In [5]:
# Load English data
english_sentences = load_data('/content/vocabs/small_vocab_en')
# Load French data
french_sentences = load_data('/content/vocabs/small_vocab_fr')
 
print('Dataset Loaded')

Dataset Loaded


In [6]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


In [7]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


In [8]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    tokenizer = Tokenizer(split=' ', char_level=False)
    tokenizer.fit_on_texts(x)
    return tokenizer.texts_to_sequences(x), tokenizer           # texts_to_sequences_generator(), Yields individual sequences.

# tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


In [9]:
text_tokenized

[[1, 2, 4, 5, 6, 7, 1, 8, 9],
 [10, 11, 12, 2, 13, 14, 15, 16, 3, 17],
 [18, 19, 3, 20, 21]]

In [10]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    if length is None:
        length = max([len(sentence) for sentence in x])
    
    return pad_sequences(x, maxlen=length, padding='post', truncating='post')

# tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


In [11]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


In [12]:
print(type(preproc_english_sentences))
print(preproc_english_sentences.shape)

<class 'numpy.ndarray'>
(137861, 15)



Models

In this section, experiment with various neural network architectures. You will begin by training four relatively simple architectures.

    Model 1 is a simple RNN
    Model 2 is a RNN with Embedding
    Model 3 is a Bidirectional RNN
    Model 4 is an optional Encoder-Decoder RNN
    Model 5 is a custom RNN (a deeper architecture that is designed to outperform all four models)

Ids Back to Text

The neural network will be translating the input to words ids, which isn't the final form we want. We want the French translation. The function logits_to_text will bridge the gab between the logits from the neural network to the French translation. You'll be using this function to better understand the output of the neural network.


In [13]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1) if index_to_words[prediction]!='<PAD>'] )

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


A basic RNN model is a good baseline for sequence data. In this model, you'll build a RNN that translates English to French.
Inputs/Outputs shape Basic RNN

    LSTM or GRU Input shape (batchsize, timesteps, 1). The timesteps is the length of the input sequences (15 for en, 21 for fr)
    output shape LSTM (batchsize, timesteps, num_units) because Return_sequence activated to return the output at each timestep
    Use of TimeDistributed wrapper over Dense(unitsoutput=target_vocab_size) to process each timestep
    for each time step, Dense outputs a probability distribution over target_vocab to predict next word


In [14]:
def token_to_words(sequence, tokenizer):
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return [index_to_words[token] for token in sequence if index_to_words[token]!='<PAD>']

In [ ]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers
    learning_rate = 0.001
    english_input = Input(shape=input_shape[1:], name="input_layer")    # the shape is (input length x 1) as batchsize excluded
    
    # LSTM takes as input (batchsize,input_length,1) and outputs (batchsize, input_length, 64) because return-seq=True
    x = LSTM(64, return_sequences=True, activation="tanh", name="LSTM_layer")(english_input)
    preds = TimeDistributed(Dense(french_vocab_size, activation="softmax"), name="Dense_layer")(x)
    model = Model(inputs=english_input, outputs=preds, name='simple_LSTM')
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

# tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))  # reshape as 3D (batchsize, timesteps, 1) for LSTM input

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

simple_rnn_model.summary()

simple_rnn_model.fit(preproc_french_sentences, tmp_x, batch_size=1024, epochs=20, validation_split=0.2)

Model: "simple_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 21, 1)]           0         
                                                                 
 LSTM_layer (LSTM)           (None, 21, 64)            16896     
                                                                 
 Dense_layer (TimeDistribute  (None, 21, 344)          22360     
 d)                                                              
                                                                 
Total params: 39,256
Trainable params: 39,256
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
108/108 [==============================] - 40s 360ms/step - loss: 3.3054 - accuracy: 0.4432 - val_loss: 2.3822 - val_accuracy: 0.4783
Epoch 2/20
108/108 [==============================] - 38s 354ms/step - loss: 2.2534 - accuracy: 0.4971 -

In [19]:
def translate(prediction, gold_standard):
    
    translation = logits_to_text(prediction[0], french_tokenizer)
    standard = ' '.join(token_to_words(gold_standard[0][:,0],french_tokenizer)) 
    print('---- Gold standard ----')
    print(standard)
    print()
    print('---- Prediction ----')
    for w_t, w_s in zip(translation.split(), standard.split()):
        if w_t == w_s:
            print('\033[0;30;0m','{}'.format(w_t), end='')
        else:
            print('\033[0;31;47m', w_t, end='')
    print()

In [ ]:
# Print prediction(s)
print('---- Original ----')
print(' '.join(token_to_words(tmp_x[:1][0][:,0],english_tokenizer) ))
print()
translate(simple_rnn_model.predict(tmp_x[:1]), preproc_french_sentences[:1])

---- Original ----
new jersey is sometimes quiet during autumn and it is snowy in april

---- Gold standard ----
new jersey est parfois calme pendant l' automne et il est neigeux en avril

---- Prediction ----
 froid est est les les et il est le en en


Model 2: Embedding (IMPLEMENTATION)
there's a better representation of a word. This is called word embeddings. An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, we create a RNN model using embedding.
Inputs/Outputs shape embedding RNN

    Input shape to Embedding layer (batchsize, input_length ie timesteps). the timesteps = length of the sequences
    Input to the Embedding layer is vector of vocab_size for each token, converted into vector of embed_dim size
    Embeddings output_shape is (None, sequence_length, embbed_dim). This is 3D expected by LSTM !
    output shape LSTM (None, timesteps = sequence_length, num_units). Return_sequence activated to return output at each timestep
    Use of TimeDistributed wrapper over Dense(target_vocab_size) to process each timestep
    for each time step, Dense outputs a probability distribution over target_vocab to predict next word



In [ ]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    
    learning_rate = 0.001
    embedding_size = 256
    
    english_input = Input(shape=input_shape[1:], name="input_layer")   # Input shape (,seq_length)
    
    embeddings = Embedding(input_dim = english_vocab_size, output_dim = embedding_size, 
                           input_length= output_sequence_length, name="Embedding_layer")(english_input)
    
    x = LSTM(64, return_sequences=True, activation="tanh", name="LSTM_layer")(embeddings)
    
    preds = TimeDistributed(Dense(french_vocab_size, activation="softmax"), name="Dense_layer")(x)
    
    model = Model(inputs=english_input, outputs=preds, name='Embedding_LSTM')
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

#tests.test_embed_model(embed_model)

# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))    # reshaped to (batchsize, seq_length) for Embedding input

# Train the neural network
embed_rnn_model = embed_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

embed_rnn_model.summary()

embed_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=20, validation_split=0.2)

Model: "Embedding_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 21)]              0         
                                                                 
 Embedding_layer (Embedding)  (None, 21, 256)          51200     
                                                                 
 LSTM_layer (LSTM)           (None, 21, 64)            82176     
                                                                 
 Dense_layer (TimeDistribute  (None, 21, 345)          22425     
 d)                                                              
                                                                 
Total params: 155,801
Trainable params: 155,801
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
108/108 [==============================] - 61s 550ms/step - loss: 3.5659 - accuracy: 0.4065 - val_los

In [ ]:
# Print prediction(s)
print('---- Original ----')
print(' '.join(token_to_words(tmp_x[:1][0],english_tokenizer) ))
print()
translate(embed_rnn_model.predict(tmp_x[:1]), preproc_french_sentences[:1])

---- Original ----
new jersey is sometimes quiet during autumn and it is snowy in april

---- Gold standard ----
new jersey est parfois calme pendant l' automne et il est neigeux en avril

---- Prediction ----
 new jersey est parfois calme au l' et il et est est en en


Model 3: Bidirectional RNNs (IMPLEMENTATION)
One restriction of a RNN is that it can't see the future input, only the past. This is where bidirectional recurrent neural networks come in. They are able to see the future data.
Inputs/Outputs shape Bidirectional RNN

    LSTM or GRU Input shape (batchsize, timesteps, 1). The timesteps is the length of the input sequences (15 for en, 21 for fr)
    output shape Bidir-LSTM (None, timesteps = sequence_length, 2 x num_units) because both outputs are concatenated.
    Return_sequence activated to return output at each timestep
    Use of TimeDistributed wrapper over Dense(target_vocab_size) to process each timestep
    for each time step, Dense outputs a probability distribution over target_vocab to predict next word



In [ ]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    
    learning_rate = 0.001
    embedding_size = 256
    
    english_input = Input(shape=input_shape[1:], name="input_layer")  # input to LSTM shape (batch, seq_length, 1)
    
    x = Bidirectional(LSTM(64, return_sequences=True, activation="tanh", name="Bidir_LSTM_layer"), input_shape=input_shape[1:])(english_input)
    
    preds = TimeDistributed(Dense(french_vocab_size, activation="softmax"), name="Dense_layer")(x)
    
    model = Model(inputs=english_input, outputs=preds, name='Bidir_LSTM')
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

#tests.test_bd_model(bd_model)

# TODO: Train and Print prediction(s)

# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2],1))  # reshape to (batch, seq_length, 1) for LSTM input

# Train the neural network
bd_rnn_model = bd_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

bd_rnn_model.summary()
    
bd_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=20, validation_split=0.2)

Model: "Bidir_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 21, 1)]           0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 21, 128)          33792     
 nal)                                                            
                                                                 
 Dense_layer (TimeDistribute  (None, 21, 345)          44505     
 d)                                                              
                                                                 
Total params: 78,297
Trainable params: 78,297
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
108/108 [==============================] - 59s 522ms/step - loss: 3.2280 - accuracy: 0.4417 - val_loss: 2.2363 - val_accuracy: 0.4857
Epoch 2/20
108/108 [===================

In [ ]:
# Print prediction(s)
print('---- Original ----')
print(' '.join(token_to_words(tmp_x[:1][0][:,0],english_tokenizer) ))
print()
translate(bd_rnn_model.predict(tmp_x[:1]), preproc_french_sentences[:1])

---- Original ----
new jersey is sometimes quiet during autumn and it is snowy in april

---- Gold standard ----
new jersey est parfois calme pendant l' automne et il est neigeux en avril

---- Prediction ----
 new jersey est parfois calme en mois et il est est est en



Model 3 bis: Embedding Bidirectional RNNs (IMPLEMENTATION)
Inputs/Outputs shape Embedding Bidirectional RNN

    Embedding requires input shape (batchsize, input_length) and outputs (batchsize, input_length, embedded_dim)
    this shape is OK for input to LSTM (3D)
    LSTM or GRU Input shape (batchsize, timesteps, embedded_dim). Return_sequence activated to return output at each timestep
    output shape Bidir-LSTM (None, timesteps = sequence_length, 2 x LSTM num_units) because both outputs are concatenated.
    Use of TimeDistributed wrapper over Dense(target_vocab_size) to process each timestep
    for each time step, Dense outputs a probability distribution over target_vocab to predict next word



In [ ]:
def emb_bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    
    learning_rate = 0.001
    embedding_size = 256
    
    english_input = Input(shape=input_shape[1:], name="input_layer")  # Embedding input (batch, seq_length)
    
    embeddings = Embedding(input_dim = english_vocab_size, output_dim = embedding_size, 
                           input_length= output_sequence_length, name="Embedding_layer")(english_input)
    
    # input shape to LSTM (batchsize, seq_length, embedding_dim) output shape: (batchsize, seq_length, units=64x2)
    x = Bidirectional(LSTM(64, return_sequences=True, activation="tanh"), name="Bidir_LSTM_layer")(embeddings)
    
    preds = TimeDistributed(Dense(french_vocab_size, activation="softmax"), name="Dense_layer")(x)
    
    model = Model(inputs=english_input, outputs=preds, name='Embedding_Bidir_LSTM')
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
   
    return model

#tests.test_bd_model(bd_model)

# TODO: Train and Print prediction(s)

# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))    # reshapped for Embedding input (batch, seq_length)

# Train the neural network
emb_bd_rnn_model = emb_bd_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

emb_bd_rnn_model.summary()

emb_bd_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=20, validation_split=0.2)

Model: "Embedding_Bidir_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 21)]              0         
                                                                 
 Embedding_layer (Embedding)  (None, 21, 256)          51200     
                                                                 
 Bidir_LSTM_layer (Bidirecti  (None, 21, 128)          164352    
 onal)                                                           
                                                                 
 Dense_layer (TimeDistribute  (None, 21, 345)          44505     
 d)                                                              
                                                                 
Total params: 260,057
Trainable params: 260,057
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
108/108 [====================

In [ ]:
# Print prediction(s)
print('---- Original ----')
print(' '.join(token_to_words(tmp_x[:1][0],english_tokenizer) ))
print()
translate(emb_bd_rnn_model.predict(tmp_x[:1]), preproc_french_sentences[:1])

---- Original ----
new jersey is sometimes quiet during autumn and it is snowy in april

---- Gold standard ----
new jersey est parfois calme pendant l' automne et il est neigeux en avril

---- Prediction ----
 new jersey est parfois calme pendant l' et il il est neigeux en avril


Model 4: Encoder-Decoder
 This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence. The decoder takes this matrix as input and predicts the translation as output.


    Encoder: One or more LSTM layers can be used to implement the encoder model. The output of this model is a fixed-size vector that represents the internal representation of the input sequence. The number of memory cells in this layer defines the length of this fixed-sized vector.

    Decoder: The decoder must transform the learned internal representation of the input sequence into the correct output sequence. One or more LSTM layers can also be used to implement the decoder model. This model reads from the fixed sized output from the encoder model.

Issue: We must connect the encoder to the decoder, and they do not fit. That is, the encoder will produce a 2-dimensional matrix of outputs, where the length is defined by the number of memory cells in the layer. The decoder is an LSTM layer that expects a 3D input of [samples, time steps, features] in order to produce a decoded sequence of some different length defined by the problem.

We can solve this using a RepeatVector layer. This layer simply repeats the provided 2D input multiple times to create a 3D output. The RepeatVector layer can be used like an adapter to fit the encoder and decoder parts of the network together. We can configure the RepeatVector to repeat the fixed length vector one time for each time step in the output sequence.



In [20]:
def encdec_model2(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # OPTIONAL: Implement
    
    learning_rate = 0.001
    embedding_size = 300
    num_units = 256
    
    ########### ENCODER ###########
    encoder_input = Input(shape=input_shape[1:], name="input_Encoder")
    
    embeddings = Embedding(input_dim = english_vocab_size, output_dim = embedding_size, 
                           input_length= input_shape[1], name="Embedding_layer")(encoder_input)
    
    encoder_output = Bidirectional(LSTM(num_units, return_sequences=False, activation="relu"), name="Bidir_Encoder")(embeddings)
    # output shape of decoder = (batch , num_units)
    
    ########### INTERMEDIARY ###########
    # This adjusts the shape of the Encoder output (2D) to the need of the Decoder (3D input).
    # We repeat the 2D vector over sequence_length times to produce the shape (batchsize, seq_length, LSTM units)
    input_decoder = RepeatVector(output_sequence_length)(encoder_output)
    
    ########### DECODER ###########
    decoder_input = Input(shape=input_decoder.shape[:1], name="input_Decoder")
    
    x = LSTM(num_units, return_sequences=True, activation="relu", name="LSTM_layer1")(input_decoder)
    x = LSTM(num_units, return_sequences=True, activation="relu", name="LSTM_layer2")(x)
    
    preds = TimeDistributed(Dense(french_vocab_size, activation="softmax"), name="Dense_layer")(x)

    ########### END-TO-END MODEL ###########
    model = Model(inputs=encoder_input, outputs=preds, name='Encdec')
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

#tests.test_encdec_model(encdec_model2) # commented out to avoid error: english sequences padded to french sequence max length

# OPTIONAL: Train and Print prediction(s)

# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))   # reshape for embedding input

# Train the neural network
enc_dec_model2 = encdec_model2(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

enc_dec_model2.summary()

enc_dec_model2.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=15, validation_split=0.2)

Model: "Encdec"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_Encoder (InputLayer)  [(None, 21)]              0         
                                                                 
 Embedding_layer (Embedding)  (None, 21, 300)          60000     
                                                                 
 Bidir_Encoder (Bidirectiona  (None, 512)              1140736   
 l)                                                              
                                                                 
 repeat_vector (RepeatVector  (None, 21, 512)          0         
 )                                                               
                                                                 
 LSTM_layer1 (LSTM)          (None, 21, 256)           787456    
                                                                 
 LSTM_layer2 (LSTM)          (None, 21, 256)           52531

In [27]:
print('---- Original ----')
print(' '.join(token_to_words(tmp_x[:1][0],english_tokenizer) ))
print()
translate(enc_dec_model2.predict(tmp_x[:1]), preproc_french_sentences[:1])

---- Original ----
new jersey is sometimes quiet during autumn and it is snowy in april

---- Gold standard ----
new jersey est parfois calme pendant l' automne et il est neigeux en avril

---- Prediction ----
 new jersey est parfois calme pendant l' et et il neigeux neigeux en



Encoder Decoder model with GRU RNN

    No bidirectional unit
    1 layer depth for both encoder and decoder
    Much less parameters compared to LSTM



In [29]:
def encdec_model4(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # OPTIONAL: Implement
    
    learning_rate = 0.001
    embedding_size = 256
    num_units = 256
    
    model = Sequential(name='enc_dec_model4')
    
    ########### ENCODER ###########
    # Embedding layer - Input shape (batch_size, seq_length)
    model.add(Embedding(input_dim = english_vocab_size, output_dim = embedding_size, 
                           input_length= input_shape[1], name="Embedding_layer"))
    
    # Expected shape for GRU layer input : (batchsize, Seq_length, embedding_size)
    model.add(GRU(num_units, go_backwards=True, name='GRU_encoder'))
    # output shape of decoder = (batch , num_units)
    
    ########### INTERMEDIARY ###########
    # This adjusts the shape of the Encoder output (2D) to the need of the Decoder (3D input).
    # We repeat the 2D vector over sequence_length times to produce the shape (batchsize, seq_length, num_units)
    model.add(RepeatVector(output_sequence_length, name='Glue'))
    # output shape of glue intermediary step = (batch , seq_length, num_units) compatible with expected input for Decoder GRU  
    
    ########### DECODER ###########
       
    model.add(GRU(num_units, return_sequences=True, name='GRU_decoder'))
    
    model.add(TimeDistributed(Dense(french_vocab_size), name='Dense'))
    
    model.add(Activation('softmax', name='softmax'))
    # output shape of decoder = (batch , french_vocab_size) probability distribution over target vocabulary
    
    ########### END-TO-END MODEL ###########
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

#tests.test_encdec_model(encdec_model) # commented out to avoid error: english sequences padded to french sequence max length

# OPTIONAL: Train and Print prediction(s)

# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length) # pad input sequence to output sequence length
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))    # reshape for embedding input

# Train the neural network
enc_dec_model4 = encdec_model4(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

enc_dec_model4.summary()

enc_dec_model4.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=30, validation_split=0.2)

Model: "enc_dec_model4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Embedding_layer (Embedding)  (None, 21, 256)          51200     
                                                                 
 GRU_encoder (GRU)           (None, 256)               394752    
                                                                 
 Glue (RepeatVector)         (None, 21, 256)           0         
                                                                 
 GRU_decoder (GRU)           (None, 21, 256)           394752    
                                                                 
 Dense (TimeDistributed)     (None, 21, 345)           88665     
                                                                 
 softmax (Activation)        (None, 21, 345)           0         
                                                                 
Total params: 929,369
Trainable params: 929,369
Non-

In [31]:
# Print prediction(s)
print('---- Original ----')
print(' '.join(token_to_words(tmp_x[:1][0],english_tokenizer) ))
print()
translate(enc_dec_model4.predict(tmp_x[:1]), preproc_french_sentences[:1])

---- Original ----
new jersey is sometimes quiet during autumn and it is snowy in april

---- Gold standard ----
new jersey est parfois calme pendant l' automne et il est neigeux en avril

---- Prediction ----
 new jersey est parfois calme au cours automne et il est il en



Model 5: Custom (IMPLEMENTATION)

Creating a model that incorporates embedding and a bidirectional rnn into one model.
Inputs/Outputs shape Customl RNN

    Embedding requires input shape (batchsize, input_length) and outputs (batchsize, input_length, embedded_dim)
    this shape is OK for input to LSTM (3D)
    LSTM or GRU Input shape (batchsize, timesteps, embedded_dim). Return_sequence activated to return output at each timestep
    output shape Bidir-LSTM (None, timesteps = sequence_length, 2 x LSTM num_units) because both outputs are concatenated.
    Use of TimeDistributed wrapper over Dense(target_vocab_size) to process each timestep
    for each time step, Dense outputs a probability distribution over target_vocab to predict next word



In [17]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    
    learning_rate = 6e-3
    embedding_size = 256
    units = 256
    
    model = Sequential()
    
    ########### ENCODER ###########
    
    model.add(Embedding(input_dim = english_vocab_size, output_dim = embedding_size, 
                           input_length= input_shape[1], name="Embedding_layer"))
    model.add(Bidirectional(LSTM(units, return_sequences=False), name='Bi_LSTM_encoder'))
    
    ########### INTERMEDIARY ###########
    
    model.add(RepeatVector(output_sequence_length, name='Glue'))
    
    ########### DECODER ###########
    
    model.add(LSTM(units, return_sequences=True, name='LSTM_decoder'))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax'), name='Dense'))
    
    ########### END-TO-END MODEL ###########
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model
#tests.test_model_final(model_final)

# TODO: Train the final model

tmp_x = pad(preproc_english_sentences, max_french_sequence_length) # pad input sequence to output sequence length
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))    # reshape for embedding input

# Train the neural network
final_model = model_final(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

final_model.summary()

final_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

print('Final Model Loaded')

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Embedding_layer (Embedding)  (None, 21, 256)          51200     
                                                                 
 Bi_LSTM_encoder (Bidirectio  (None, 512)              1050624   
 nal)                                                            
                                                                 
 Glue (RepeatVector)         (None, 21, 512)           0         
                                                                 
 LSTM_decoder (LSTM)         (None, 21, 256)           787456    
                                                                 
 Dense (TimeDistributed)     (None, 21, 345)           88665     
                                                                 
Total params: 1,977,945
Trainable params: 1,977,945
Non-trainable params: 0
____________________________________________

In [20]:
print('---- Original ----')
print(' '.join(token_to_words(tmp_x[:1][0],english_tokenizer) ))
print()
translate(final_model.predict(tmp_x[:1]), preproc_french_sentences[:1])

---- Original ----
new jersey is sometimes quiet during autumn and it is snowy in april

---- Gold standard ----
new jersey est parfois calme pendant l' automne et il est neigeux en avril

---- Prediction ----
 new jersey est parfois calme pendant l' automne et il est neigeux en avril



Prediction (IMPLEMENTATION)

    the objective is to reach a minimum of 95% validation accuracy so that to have meaningful predictions



In [21]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # align padded english sequence to the length of french padded length
    x = pad_sequences(x)     
    
    # Instantiate the best model

    model = model_final(x.shape,
                        preproc_french_sentences.shape[1],
                        len(english_tokenizer.word_index)+1,
                        len(french_tokenizer.word_index)+1)
    
    model.summary()
                                 
    # train the model - Reach a minimum of 94% accuracy so that to have meaningful predictions
    model.fit(x, y, batch_size=1024, epochs=10, validation_split=0.2)

    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))

final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Embedding_layer (Embedding)  (None, 15, 256)          51200     
                                                                 
 Bi_LSTM_encoder (Bidirectio  (None, 512)              1050624   
 nal)                                                            
                                                                 
 Glue (RepeatVector)         (None, 21, 512)           0         
                                                                 
 LSTM_decoder (LSTM)         (None, 21, 256)           787456    
                                                                 
 Dense (TimeDistributed)     (None, 21, 345)           88665     
                                                                 
Total params: 1,977,945
Trainable params: 1,977,945
Non-trainable params: 0
____________________________________________